In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv("../Notebooks/mushedi.csv")
df0 = pd.read_csv("../Notebooks/mushpoi.csv")

In [3]:
df1['Class'] = 1
df0['Class'] = 0
df = pd.concat([df1, df0], axis=0)
df
for i in df.columns:
    print(df[i].name, df[i].nunique())


capshp 6
capsurf 4
capcol 8
buis 2
odor 7
gilatt 2
gillspac 2
gillsiz 2
gillcol 9
stlkshp 2
stlkroot 4
surfabovring 4
surfbeloring 4
colabovring 7
colbeloring 7
veiltyp 1
veilcol 2
ringnum 3
ringtyp 4
sporcol 6
pop 6
hab 6
Class 2


In [4]:
df1.shape

(3488, 23)

In [5]:
df0.shape

(2156, 23)

In [6]:
from sklearn.utils import resample
df1_downsamp = resample(df1,
                       replace=True,
                       n_samples=len(df0),
                       random_state=49209)

print(df1_downsamp.shape)

(2156, 23)


In [7]:
df = pd.concat([df1_downsamp, df0], axis=0)

In [8]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1]

y = df.Class

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, 
                                                    random_state = 49209,
                                                    stratify = y)

In [9]:
X_train

,capshp,capsurf,capcol,buis,odor,gilatt,gillspac,gillsiz,gillcol,stlkshp,...,surfbeloring,colabovring,colbeloring,veiltyp,veilcol,ringnum,ringtyp,sporcol,pop,hab
960,x,s,w,f,n,f,w,b,h,t,...,f,w,w,p,w,o,e,n,a,g
2428,x,y,e,t,n,f,c,b,w,t,...,s,p,g,p,w,o,p,k,v,d
1905,x,f,g,t,n,f,c,b,p,t,...,s,g,p,p,w,o,p,n,v,d
982,x,f,y,f,f,f,c,b,h,e,...,k,b,b,p,w,o,l,h,y,g
64,x,y,w,t,p,f,c,n,w,e,...,s,w,w,p,w,o,p,k,v,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,x,y,g,f,f,f,c,b,g,e,...,k,b,n,p,w,o,l,h,v,d
2042,x,f,n,t,n,f,c,b,w,t,...,s,g,p,p,w,o,p,n,y,d
1740,x,f,n,t,n,f,c,b,p,t,...,s,w,w,p,w,o,p,k,y,d
1283,f,y,y,f,f,f,c,b,h,e,...,k,n,n,p,w,o,l,h,v,p


In [10]:
import numpy as np
import scipy
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

param_range = scipy.stats.loguniform(0.0001, 1000.0)

np.random.seed(1)
param_range.rvs(10)

pipe_svc = make_pipeline(OneHotEncoder(handle_unknown='ignore'),
                         StandardScaler(with_mean = False),
                         LinearSVC())

param_grid = [{'linearsvc__C': param_range}]

rs = RandomizedSearchCV(estimator=pipe_svc,
                        param_distributions=param_grid,
                        scoring='accuracy',
                        refit=True,
                        cv=10,
                        random_state=1,
                        n_jobs=1)


rs = rs.fit(X_train, y_train)
y_pred = rs.predict(X_test)
print("Best accuracy and model C: Base data, non-PCA")
print(rs.best_score_)
print(rs.best_params_)

Best accuracy and model C: Base data, non-PCA
0.9994202898550725
{'linearsvc__C': np.float64(0.08301451461243861)}


In [11]:
from sklearn.metrics import classification_report
y_true = y_test
target_names = ['class 0 (Poison)', 'class 1 (Edible)']
print("Classification Report: Base data, non-PCA")
print(classification_report(y_true, y_pred, target_names=target_names))

Classification Report: Base data, non-PCA
                  precision    recall  f1-score   support

class 0 (Poison)       1.00      1.00      1.00       432
class 1 (Edible)       1.00      1.00      1.00       431

        accuracy                           1.00       863
       macro avg       1.00      1.00      1.00       863
    weighted avg       1.00      1.00      1.00       863



In [13]:
X_train = X_train.drop(columns=['capsurf', 'pop', 'gillsiz'])
X_test = X_test.drop(columns=['capsurf', 'pop', 'gillsiz'])

In [14]:
import numpy as np
import scipy
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

param_range = scipy.stats.loguniform(0.0001, 1000.0)

np.random.seed(1)
param_range.rvs(10)

pipe_svc = make_pipeline(OneHotEncoder(handle_unknown='ignore'),
                         StandardScaler(with_mean = False),
                         LinearSVC())

param_grid = [{'linearsvc__C': param_range}]

rs = RandomizedSearchCV(estimator=pipe_svc,
                        param_distributions=param_grid,
                        scoring='accuracy',
                        refit=True,
                        cv=10,
                        random_state=1,
                        n_jobs=1)


rs = rs.fit(X_train, y_train)
y_pred = rs.predict(X_test)
print("Best accuracy and model C: Base data, non-PCA")
print(rs.best_score_)
print(rs.best_params_)


from sklearn.metrics import classification_report
y_true = y_test
target_names = ['class 0 (Poison)', 'class 1 (Edible)']
print("Classification Report: Base data, non-PCA")
print(classification_report(y_true, y_pred, target_names=target_names))

Best accuracy and model C: Base data, non-PCA
1.0
{'linearsvc__C': np.float64(0.08301451461243861)}
Classification Report: Base data, non-PCA
                  precision    recall  f1-score   support

class 0 (Poison)       1.00      1.00      1.00       432
class 1 (Edible)       1.00      1.00      1.00       431

        accuracy                           1.00       863
       macro avg       1.00      1.00      1.00       863
    weighted avg       1.00      1.00      1.00       863

